In [1]:
import numpy as np
import sys
from functools import partial
import os
import tritongrpcclient
import tritongrpcclient.model_config_pb2 as mc
import tritonhttpclient
from tritonclientutils import triton_to_np_dtype
from tritonclientutils import InferenceServerException
import torch
import torch.nn.functional as F
from tqdm import tqdm
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
import os
from torch_geometric.utils import to_networkx
import networkx as nx
# importing matplotlib.pyplot
import matplotlib.pyplot as plt
import numpy as np

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/tritongrpcclient/__init__.py:30: DeprecationWarning: The package `tritongrpcclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.grpc`
  warnings.warn(
/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/tritongrpcclient/model_config_pb2.py:30: DeprecationWarning: The package `tritongrpcclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.grpc.model_config_pb2`
  warnings.warn(
/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/tritonhttpclient/__init__.py:30: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(
/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/tritonclientutils/__init__.py:30: DeprecationWarning: The package `tritonclientutils` is deprecated and will b

In [3]:
root = '/Users/sachin/Desktop/arangodb/scripts/ArangoML/graph_embeddings/products'

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
dataset = PygNodePropPredDataset('ogbn-products', root)

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# getting train val test split idx based on sales ranking
split_idx = dataset.get_idx_split()
evaluator = Evaluator(name='ogbn-products')
data = dataset[0]

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
test_idx = split_idx['test']


/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
test_loader = NeighborSampler(data.edge_index, node_idx=test_idx,
                              sizes=[15, 10, 5], batch_size=1,
                              shuffle=False, num_workers=12)

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
# selecting random node for input
dummy_n_ids = []
dummy_adjs = []
for idx, (batch_size, n_id, adjs) in enumerate(test_loader):
        if idx == 100:
            dummy_n_ids.append(n_id)
            dummy_adjs.append(adjs)
            break

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
# creating adjs for performing a trace on the GraphSage model
# will contain only edge_idx and size attributes
edge_list_0 = []
edge_list_1 = []
edge_list_2 = []
edge_adjs = []
for idx, e_idx in enumerate(dummy_adjs[0]):
    if idx == 0:
        edge_list_0.append(e_idx[0])
        #edge_list_0.append(e_idx[1])
        edge_list_0.append(torch.tensor(np.asarray(e_idx[2])))
    elif idx == 1:
        edge_list_1.append(e_idx[0])
        #edge_list_1.append(e_idx[1])
        edge_list_1.append(torch.tensor(np.asarray(e_idx[2])))
    else:
        edge_list_2.append(e_idx[0])
        #edge_list_2.append(e_idx[1])
        edge_list_2.append(torch.tensor(np.asarray(e_idx[2])))

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
# loading edge_index and size layer wise
edge_index_0 = edge_list_0[0]
edge_size_0 = edge_list_0[1]

edge_index_1 = edge_list_1[0]
edge_size_1 = edge_list_1[1]

edge_index_2 = edge_list_2[0]
edge_size_2 = edge_list_2[1]

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
# loading node feature matrix
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
# lets create node dummy input for the trace
dummy_x = x[dummy_n_ids[0]]
dummy_x.shape

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


torch.Size([459, 100])

In [60]:
# pad nodes
max_nodes = 1000
total_nodes = dummy_x.size(0)
nodes_padded = max_nodes - total_nodes
dummy_x_pad = F.pad(input=dummy_x, pad=(0, 0, 0, nodes_padded), mode='constant', value=0)

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [61]:
dummy_x_pad.shape

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


torch.Size([1000, 100])

## Let's test prediction on  test nodes 

In [62]:
input_name = ['input__0', 'input__1', 'input__2', 'input__3', 'input__4', 'input__5', 'input__6']
output_name = ['output__0', 'output__1', 'output__2']
VERBOSE = False
from tritonclient.utils import *

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
def run_inference(node_matrix, edge_index_0, edge_size_0, edge_index_1, edge_size_1, edge_index_2, edge_size_2, model_name='graph_embeddings', url='127.0.0.1:8000', model_version='1'):
    triton_client = tritonhttpclient.InferenceServerClient(
        url=url, verbose=VERBOSE)
    model_metadata = triton_client.get_model_metadata(
        model_name=model_name, model_version=model_version)
    model_config = triton_client.get_model_config(
        model_name=model_name, model_version=model_version)
    # I have restricted the input sequence length to 256

    input_node_matrix = node_matrix
    input_node_matrix  = np.array(input_node_matrix, dtype=np.float32)
    
    # edges_indexes and sizes
    ed_index_0 = np.array(edge_index_0, dtype=np.int64)
    ed_index_1 = np.array(edge_index_1, dtype=np.int64)
    ed_index_2 = np.array(edge_index_2, dtype=np.int64)
    
    ed_size_0 = np.array(edge_size_0, dtype=np.int64)
    ed_size_1 = np.array(edge_size_1, dtype=np.int64)
    ed_size_2 = np.array(edge_size_2, dtype=np.int64)
    


    input0 = tritonhttpclient.InferInput(input_name[0], (1000,  100), 'FP32')
    input0.set_data_from_numpy(input_node_matrix, binary_data=False)
    
    #layer-1
    input1 = tritonhttpclient.InferInput(input_name[1], ed_index_0.shape, 'INT64')
    input1.set_data_from_numpy(ed_index_0, binary_data=False)
    #size
    input2 = tritonhttpclient.InferInput(input_name[2], (2,), 'INT64')
    input2.set_data_from_numpy(ed_size_0, binary_data=False)
    
    #layer-2
    input3 = tritonhttpclient.InferInput(input_name[3], ed_index_1.shape, 'INT64')
    input3.set_data_from_numpy(ed_index_1, binary_data=False)
    #size
    input4 = tritonhttpclient.InferInput(input_name[4], (2,), 'INT64')
    input4.set_data_from_numpy(ed_size_1, binary_data=False)
    
    #layer-3
    input5 = tritonhttpclient.InferInput(input_name[5], ed_index_2.shape, 'INT64')
    input5.set_data_from_numpy(ed_index_2, binary_data=False)
    #size
    input6 = tritonhttpclient.InferInput(input_name[6], (2,), 'INT64')
    input6.set_data_from_numpy(ed_size_2, binary_data=False)
    
    output0 = tritonhttpclient.InferRequestedOutput(output_name[0],  binary_data=False)
    output1 = tritonhttpclient.InferRequestedOutput(output_name[1],  binary_data=False)
    output2 = tritonhttpclient.InferRequestedOutput(output_name[2],  binary_data=False)
    
    response = triton_client.infer(model_name, model_version=model_version, inputs=[input0, input1, input2, input3, input4, input5, input6], outputs=[output0, output1, output2])
    # layer-1 embeddings
    embeddings_layer_1 = response.as_numpy('output__0')
    # layer-2 embeddings
    embeddings_layer_2 = response.as_numpy('output__1')
    # # layer-3 embeddings
    embeddings_layer_3 = response.as_numpy('output__2')
    return embeddings_layer_1, embeddings_layer_2, embeddings_layer_3

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
emb1, emb2, emb3 = run_inference(dummy_x_pad, edge_index_0, edge_size_0, edge_index_1, edge_size_1, edge_index_2, edge_size_2)

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
emb3.shape

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(500, 47)

In [66]:
emb3[0]

/Users/sachin/anaconda3/envs/transformers/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([  4.2246513 ,  -0.4564677 ,  -2.1890893 ,   1.6496025 ,
         3.010801  ,  -2.2272007 ,   5.33481   ,  -9.296241  ,
         0.4352603 ,  -5.7606072 ,   5.416895  ,  -0.32884753,
        -3.7443688 ,  -5.3133497 , -11.545118  ,  -9.265142  ,
         7.892888  ,   3.3840961 ,   0.16178823,  -1.949775  ,
       -12.229473  ,   0.7407882 ,  -9.48527   ,  -2.15074   ,
        -5.0011497 ,  -1.7124702 ,  -4.353361  ,  -2.8487844 ,
        -4.9935174 , -23.178326  ,  -5.5515566 ,  -7.5406933 ,
        -8.372204  ,  -9.640902  , -15.568034  , -16.474665  ,
        -7.3028746 ,  -8.771542  , -10.82653   ,  -7.845075  ,
       -11.450493  , -10.724498  , -10.494268  , -10.281946  ,
        -9.315411  ,  -9.914256  , -10.783257  ], dtype=float32)